# NLP (comentários página do Facebook)

In [157]:
import pandas as pd
import nltk
import re
import string
import unicodedata
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn import svm

In [158]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [159]:
def normalize_accents(text):
    return unicodedata.normalize("NFKD", text).encode("ASCII", "ignore").decode("utf-8")

def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

def normalize_str(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = normalize_accents(text)
    text = re.sub(re.compile(r" +"), " ",text)
    return " ".join([w for w in text.split()])

def tokenizer(text):
    stop_words = stopwords.words("portuguese") # portuguese, caso o dataset seja em português
    if isinstance(text, str):
        text = normalize_str(text)
        text = "".join([w for w in text if not w.isdigit()])
        text = word_tokenize(text)
        text = [x for x in text if x not in stop_words]
        text = [y for y in text if len(y) > 2]
        return " ".join([t for t in text])
    else:
        return None

In [160]:
df_1 = pd.read_csv('/content/chatgpt-base-comentarios-negativos.csv', sep=';')
df_2 = pd.read_csv('/content/chatgpt-base-comentarios-neutros.csv', sep=';')
df_3 = pd.read_csv('/content/chatgpt-base-comentarios-positivos.csv', sep=';')
df_4 = pd.read_csv('/content/facebook-mentions.csv', sep=';')
df_5 = pd.read_csv('/content/facebook-reviews.csv', sep=';')
df = pd.concat([df_1, df_2, df_3, df_4, df_5])
df

,texto,categoria
0,O atendimento foi péssimo.,negativo
1,Produto de baixa qualidade.,negativo
2,Muito insatisfeito com a compra.,negativo
3,Demorou muito para chegar.,negativo
4,Chegou quebrado e não funciona.,negativo
...,...,...
7,De projeto...uma ONG! Exemplo para muitas pess...,positivo
8,Projeto sério realizado por uma equipe extrema...,positivo
9,“A educação é a arma mais poderosa que você po...,positivo
10,UM exemplo de instituição... estão de parabéns...,positivo


In [161]:
df['categoria'].unique()

array(['negativo', 'neutro', 'positivo'], dtype=object)

In [162]:
df.shape

(1060, 2)

In [163]:
df.head()

,texto,categoria
0,O atendimento foi péssimo.,negativo
1,Produto de baixa qualidade.,negativo
2,Muito insatisfeito com a compra.,negativo
3,Demorou muito para chegar.,negativo
4,Chegou quebrado e não funciona.,negativo


In [164]:
df = shuffle(df)
df = df.reset_index(drop=True)

In [165]:
df.head()

,texto,categoria
0,Atendimento de qualidade.,positivo
1,Produto de qualidade superior.,positivo
2,Recebi um produto usado.,negativo
3,"Apoio os objetivos da ONG, todavia, sinto que ...",neutro
4,A ONG oferece um feedback excelente aos doador...,positivo


In [166]:
df['texto_processado'] = df['texto'].apply(tokenizer)
df.head()

,texto,categoria,texto_processado
0,Atendimento de qualidade.,positivo,atendimento qualidade
1,Produto de qualidade superior.,positivo,produto qualidade superior
2,Recebi um produto usado.,negativo,recebi produto usado
3,"Apoio os objetivos da ONG, todavia, sinto que ...",neutro,apoio objetivos ong todavia sinto falta recurs...
4,A ONG oferece um feedback excelente aos doador...,positivo,ong oferece feedback excelente doadores sempre...


In [167]:
X = df['texto_processado'].values
y = df['categoria'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=777, stratify=y)

In [168]:
print('X_train:', X_train.size)
print('X_test:', X_test.size)
print('y_train:', y_train.size)
print('y_test:', y_test.size)

X_train: 742
X_test: 318
y_train: 742
y_test: 318


In [169]:
vect = CountVectorizer(lowercase=False)
vect.fit(X_train)

CountVectorizer(lowercase=False)

In [170]:
X_train = vect.transform(X_train)
X_train

<742x685 sparse matrix of type '<class 'numpy.int64'>'
	with 3695 stored elements in Compressed Sparse Row format>

In [171]:
X_test = vect.transform(X_test)
X_test

<318x685 sparse matrix of type '<class 'numpy.int64'>'
	with 1516 stored elements in Compressed Sparse Row format>

In [172]:
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# TODO: talvez utilizar um xgboost? ou rodar o cross_val para validar varios modelos em paralelo

In [173]:
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.7484276729559748


In [174]:
texto_teste = tokenizer('Não gostei da ONG, não prestaram o serviço de forma adequada.')
text_vect = vect.transform([texto_teste])
clf.predict(text_vect)

array(['negativo'], dtype=object)

In [175]:
texto_teste = tokenizer('Gostei da ONG!')
text_vect = vect.transform([texto_teste])
clf.predict(text_vect)

array(['positivo'], dtype=object)

In [176]:
texto_teste = tokenizer('A OGN poderia oferecer mais serviços, mas ok.')
text_vect = vect.transform([texto_teste])
clf.predict(text_vect)

array(['neutro'], dtype=object)

In [180]:
texto_teste = tokenizer('Parece uma boa ONG, contudo poderia ser mais transparente!')
text_vect = vect.transform([texto_teste])
clf.predict(text_vect)

array(['neutro'], dtype=object)